In [27]:
%matplotlib inline
from __future__ import division
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
import os
from matplotlib.colors import ListedColormap
import matplotlib.colors
import time

from PIL import Image, ImageDraw

In [28]:
class Ising():
    def __init__(self, temp, n, msrmnt):
        self.temp = temp
        self.n = n
        self.msrmnt = msrmnt

    ''' Simulating the Ising model '''    
    ## monte carlo moves
    def mcmove(self, config, N, beta):
        ''' This is to execute the monte carlo moves using 
        Metropolis algorithm such that detailed
        balance condition is satisified'''
        #randomStarts = np.random.randint(N*N, size = N*N)
        aArray = np.random.randint(n, size = N*N)
        bArray = np.random.randint(n, size = N*N)
        for i in range(N*N):       
            '''
            a = np.random.randint(0, N)
            b = np.random.randint(0, N)
            '''
            #s = config[a[i], a[b]]

            a = aArray[i]
            b = bArray[i]
            s =  config[a, b]
            nb = config[(a+1)%N,b] + config[a,(b+1)%N] + config[(a-1)%N,b] + config[a,(b-1)%N] + config[(a+1)%N, (b-1)%N] + config[(a-1)%N, (b+1)%N]
            cost = 2*s*nb
            if cost < 0:	
                s *= -1
            elif rand() < np.exp(-cost*beta):
                s *= -1
            config[a, b] = s
            #config[i, j] = s
        return config
    
    def simulate(self):   
        ''' This module simulates the Ising model'''
#***MODIFY THIS TO CONVERT self.temp TEMPERATURE to thermodynamic beta*** 
        temp  = self.temp
        N     = self.n        # Initialise the lattice
        msrmnt = self.msrmnt
        config = 2*np.random.randint(2, size=(N,N))-1
        f = plt.figure(figsize=(15, 15), dpi=80);    
        #self.configPlot(f, config, 0, N, 1);
        # naming convention is path/Ising_<temperature>
        plotpath = ""
        plotname = [plotpath,"Ising_",str(self.temp),"_"]
        plotname = "".join(plotname)
        i = 0
        while os.path.exists(f"{plotname}{i}.png"):
            i += 1
        plotname = [plotname,str(i),".png"]
        print(plotname)
        plotname = "".join(plotname)
        
        for i in range(msrmnt+1):
            start_time = time.time()
            self.mcmove(config, N, 1.0/(temp))
            end_time = time.time() - start_time
            print("iteration: " + str(i) + " ---- time taken: " + str(end_time))
            if i == msrmnt:    
                self.configPlot(f, config, i, N, 1);
                
        f.savefig(plotname, transparent = True, bbox_inches = 'tight', pad_inches = 0)
        makeHexagonImage(N, config, plotname)
        plt.close(f)
                    
    def configPlot(self, f, config, i, N, n_):
        ''' This modules plts the configuration once passed to it along with time etc '''
        X, Y = np.meshgrid(range(N+1), range(N+1))
        sp =  f.add_subplot(3, 3, n_ )  
        plt.setp(sp.get_yticklabels(), visible=False)
        plt.setp(sp.get_xticklabels(), visible=False)      
        plt.gca().set_axis_off()
        plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
        plt.margins(0,0)
        #this is ridiculously suboptimal but i see no other way to do this with cmaps
        #extra N^2 operations for no real reason
        z = 0
        for i in range(N):
            for j in range(N):
                if config[i][j] != 1:
                    z = 1
        
        if z:
            plt.pcolormesh(X, Y, config, cmap=plt.cm.RdBu);
        else:
            bluemap = ListedColormap((0.019, 0.188, 0.380, 1))
            plt.pcolormesh(X, Y, config, cmap=bluemap);
    

    

In [29]:
from PIL import Image, ImageDraw, ImageOps

def makeHexagonImage(n, config, plotname):
    img = Image.new("RGB", (2*n, 3*n))
    imred = Image.new("RGB", (2, 2), color = (103, 0, 31))
    imbl = Image.new("RGB", (2, 2), color = (5, 48, 97))

    #print(n)
    for x in range(n):
        for y in range(n):
            if config[x, y] == 1:
                im = imbl #blue is 1
            else:
                im = imred
            img.paste(im, (2*y, 2*x + y,2*y+2 , 2*x + y + 2))
    img = ImageOps.flip(img)
    img.save("hex"+plotname) #make better naming system


In [30]:
# temp_dist is the input temperature distribution
temp_dist = np.array([0.1, 0.5, 1, 2, 3, 4, 5, 6, 10, 16, 20]) #0.05 removed temporarily
# n is resolution of image (nxn)
n = 1024
# Final iteration/time
msrmnt = 2000 #this can probably be lower right
'''
for x in range(3):
    for temp in temp_dist:
        rm = Ising(temp, n, msrmnt)
        rm.simulate()
'''
for x in range(20):
    rm = Ising(0.00001, n, msrmnt)
    rm.simulate()

['Ising_1e-05_', '0', '.png']


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()